In [1]:
## SCOT_Event_Related ##

## SCOT Processing Pipeline created by Neil Dundon and adapted by Tom Bullock to visualize and output event-codes/triggers
## Date: 09.20.23

## CELL 1: LOAD ###
# Run this cell to load your datafile through the GUI (currently only works with .acq files, any template).
# For each window, select the required option then press click "X" to close the window (your selection will be saved) 
# (1) Select the channel containing acceleration.
# (2) Select the channel containing respiration (either resp belt or resp. estimated from z0 timeseries using low-pass filter). 
# (3) Is there an event channel present?
# (4) If there is an event channel present, select the channel.

# NOTES: Currently getting "ImportError: cannot import name md5" warning messages, but these don't seem prevent the datafile from being loaded successfully.  

!pip install bioread
from sys_SCOT_Event_Related import loadem
cont_dict,file_path=loadem()
print('data loaded!')

ERROR:root:code for hash md5 was not found.
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 147, in <module>
    globals()[__func_name] = __get_hash(__func_name)
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 97, in __get_builtin_constructor
    raise ValueError('unsupported hash type ' + name)
ValueError: unsupported hash type md5
ERROR:root:code for hash sha1 was not found.
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 147, in <module>
    globals()[__func_name] = __get_hash(__func_name)
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 97, in __get_builtin_constructor
    raise ValueError('unsupported hash type ' + name)
ValueError: unsupported hash 

2023-09-19 18:29:06.898 python[81813:10554618] +[CATransaction synchronize] called within transaction


selected contractility channel is Acc dZ (40Hz LP)
selected respiration channel is Respiration
selected event channel is Experiment Trigger
extracting raw signals from AcqKnowledge files...
removing slow movements from acceleration data with 7-order polynomial
applying lowpass filter to acceleration channel, 22.5 Hz cutoff
removing slow movements from respiration data with 7-order polynomial
applying lowpass filter to respiration channel, 0.35 Hz cutoff
estimating respiration amount and cycle...
data loaded!


In [2]:
## CELL 2: SET THRESHOLD AND ESTIMATE PEAKS ###
# Run this cell to open a plot to decide minimum peak height in the acceleration timeseries.
# Enter value (will be around ~0.5) in the accompanying GUI.
# Programme will then do its first estimate on peak values.

import matplotlib.pyplot as plt
from sys_SCOT_Event_Related import compute_peaks,thresh_ind

%matplotlib tk
plt.close('all')
i=thresh_ind(cont_dict['hz'])
plt.plot(cont_dict['t'][i],cont_dict['s'][i])
peak_times,peak_vals,peak_inds=compute_peaks(cont_dict)

plt.close('all')

In [3]:
## CELL 3: INTERACTIVE PLOT - PEAKS IN ACCELERATION TIME SERIES ###
# Run this cell to open the interactive plot to visualise and amend peaks in the acceleration time series.
# Accleration peaks are more robust to noise and are better estimates of heartbeat times.
# Left click - add peak.
# Right click - remove peak.
# Hold 'm' and left click - use moving average to estimate peak.

# Note that the Custom Event Codes section just allows you to input arrays of event codes that mark the beginning and
# end of recording blocks.  These event codes are then visualized when you open the interactive plot, allowing you to 
# clearly see where recording blocks start and end. This can be useful, for example, if you have one long continuous
# recording file containing multiple blocks of a task separated by break periods containing data that you won't be 
# analyzing.  

# Note: Edit the custom event codes section to parse 

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from sys_SCOT_Event_Related import *

%matplotlib tk
plt.close('all')
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

w = ax.plot(cont_dict['t'],cont_dict['s'])
peak_plot, = ax.plot(peak_times,peak_vals ,'vm')


## CUSTOM EVENT-CODE VISUALIZATION START HERE ##

# isolate event codes and timestamps from dict
events = cont_dict['events_s']
times = cont_dict['events_t']
    
code_mat = []
time_mat = []

# cut out redundant events from oversampling
for i in np.arange(np.shape(events)[0]): 
    if events[i] != 0 and events[i-1] != events[i]:       
        code_mat.append(str(int(events[i])))
        time_mat.append(round(times[i],3))
        
# find start of steady state blocks and then calculate end of blocks so we can insert end triggers [note that when we 
# collected the data we did not send event codes to mark the end of blocks; however, given that we know each block lasts
# 90 seconds, we can just create a vector of block end event codes and times here ]
start_block_event_codes_ssvep = ['11','12', '13', '21', '22', '23', '31','32','33','41','42','43', '51','52','53'] # 
start_block_event_times_ssvep = []
end_block_event_times_ssvep = []
for i in np.arange(len(start_block_event_codes_ssvep)):
    event = start_block_event_codes_ssvep[i]
    this_time = time_mat[code_mat.index(event)]    
    start_block_event_times_ssvep.append(this_time)
    end_block_event_times_ssvep.append(this_time + 90) 

end_block_event_codes_ssvep = ['911','912', '913', '921', '922', '923', '931','932','933','941','942','943', '951','952','953'] #,

# create list of start codes and times for resting state blocks
start_block_event_codes_rest = ['58','61','62','63','64','65','81','82','83','84','85'] 
start_block_event_times_rest = []
for i in np.arange(len(start_block_event_codes_rest)):    
    event = start_block_event_codes_rest[i]
    this_time = time_mat[code_mat.index(event)]
    start_block_event_times_rest.append(this_time)
    
# creae list of end codes and times for resting state blocks [these block end codes were logged in the original recording]
end_block_event_codes_rest = ['59','71','72','73','74','75','91','92','93','94','95'] 
end_block_event_times_rest = []
for i in np.arange(len(end_block_event_codes_rest)):
    event = end_block_event_codes_rest[i]
    this_time = time_mat[code_mat.index(event)]
    end_block_event_times_rest.append(this_time)

# merge lists of codes and times 
start_block_codes = start_block_event_codes_ssvep + start_block_event_codes_rest
start_block_times = start_block_event_times_ssvep + start_block_event_times_rest
end_block_codes = end_block_event_codes_ssvep + end_block_event_codes_rest
end_block_times = end_block_event_times_ssvep + end_block_event_times_rest

print(sorted(start_block_times))

## EVENTS CUSTOM CODE ENDS HERE ##


# plot green lines for start block codes and red lines for end block codes
plt.vlines(x=start_block_times,ymin=-5,ymax=5,linestyles=':',colors = 'g',linewidth=4)
plt.vlines(x=end_block_times,ymin=-5,ymax=5,linestyles=':',colors = 'r',linewidth=4)

# Set the axis and slider position in the plot
bottom_pos = plt.axes([0.2, 0.1, 0.65, 0.03],facecolor = 'white')
scroll_slider = Slider(bottom_pos,'time', -1,cont_dict['t'][-1])

# Make a vertically oriented slider to control the amplitude
right_pos = plt.axes([.95, 0.25, 0.0225, 0.63])
y_zoom = Slider(right_pos,label="Y zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")
left_pos = plt.axes([.05, 0.25, 0.0225, 0.63])
x_zoom = Slider(left_pos,label="X zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")

ax.set_ylim(-np.median(peak_vals)*4*.5,np.median(peak_vals)*4*.5)

ax.set_xlim(-1,9)

orig_n = len(peak_times)

ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(orig_n))

def update(val):
    pos = scroll_slider.val
    yzoom = y_zoom.val
    xzoom = x_zoom.val
    ax.set_xlim(pos, pos+20*xzoom)
    fig.canvas.draw_idle
    ecc=np.median(peak_vals)*4*yzoom
    ax.set_ylim(-ecc, ecc)
    fig.canvas.draw_idle
    
# update function called using on_changed() function
scroll_slider.on_changed(update)
y_zoom.on_changed(update)
x_zoom.on_changed(update)

# Display the plot
plt.show()

def onclick(event):
    global peak_times,peak_vals,fig,peak_plot,orig_n,ax
    ix, iy, ib, ik, iax = event.xdata, event.ydata, event.button, event.key, event.inaxes
    if (ix!=None) & (iax==ax):
        if (np.min(np.abs(ix-peak_times))<100) & (event.button!=1) & (ik==None):
            peak_times,peak_vals,peak_plot=remove_point(ix,peak_times,peak_vals,peak_plot)
            ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(len(peak_vals)))
        elif (event.button==1) & (ik==None) & (ix>0) & (ix>0) & (iy<np.max(peak_vals)*1.2): #update dist from pos
            peak_times,peak_vals,peak_plot=add_point(ix,iy,peak_times,peak_vals,peak_plot)
            ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(len(peak_vals)))
        elif (event.button==1)&(ik=='m'):
            peak_times,peak_vals,peak_plot=meap_dat(ix,iy,peak_times,peak_vals,peak_plot)
            ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(len(peak_vals)))
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        if not plt.fignum_exists(1):
            canvas.mpl_disconnect(cid)
            canvas.mpl_disconnect(cidk)
    return peak_times,peak_vals

def onarrow(e):
    global scroll_slider
    if e.key == "right":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]+.500)
    elif e.key == "left":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]-.500)
    else:
        return

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cidk = fig.canvas.mpl_connect('key_press_event', onarrow)

[1747.329, 2271.483, 2503.502, 2678.979, 2786.883, 2909.634, 3448.862, 3666.186, 3778.86, 3886.684, 3993.042, 4615.701, 4831.687, 4941.694, 5058.899, 5171.702, 5714.742, 5931.553, 6036.426, 6146.497, 6252.118, 6781.301, 6996.68, 7108.437, 7218.177, 7319.317]


In [4]:
## CELL 4: INTERACTIVE PLOT - FINAL CHECK - PEAKS IN CONTRACTILITY TIME SERIES###
# run this cell to open the interactive plot to visualise and amend peak amplitudes in the contractility time series
# you can no longer remove and add peaks to the dataset, only adjust their amplitudes
# left click - adjust amplitude
# hold m and left click - use moving average to estimate peak amplitude

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from sys_SCOT_Event_Related import *

##Store acceleration peaks and times
cont_dict['acc_peaks']=peak_vals
cont_dict['acc_peak_times']=peak_times

%matplotlib tk
plt.close('all')
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

#estimating heartrate before estimating contractility
cont_dict['RR']=np.hstack([np.nan,np.diff(peak_times)])

#estimate contractility from acceleration and adjust times etc.
cont_s = np.diff(cont_dict['s'])
cont_t = cont_dict['t'][1:]
init_cont_inds = peak_inds+1
cont_inds = shift_peaks_no_plot(250,peak_times,cont_t,cont_s,init_cont_inds)
peak_times = cont_t[cont_inds]
peak_vals = cont_s[cont_inds]

w = ax.plot(cont_t,cont_s)
peak_plot, = ax.plot(peak_times,peak_vals ,'vm')

# Set the axis and slider position in the plot
bottom_pos = plt.axes([0.2, 0.1, 0.65, 0.03],facecolor = 'white')
scroll_slider = Slider(bottom_pos,'time', -1,cont_t[-1])

# Make a vertically oriented slider to control the amplitude
right_pos = plt.axes([.95, 0.25, 0.0225, 0.63])
y_zoom = Slider(right_pos,label="Y zoom",valmin=0,valmax=.1,valinit=.05,orientation="vertical")
left_pos = plt.axes([.05, 0.25, 0.0225, 0.63])
x_zoom = Slider(left_pos,label="X zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")

ax.set_ylim(-(np.median(peak_vals)+.05),np.median(peak_vals)+.05)

ax.set_xlim(-1,9)

ax.set_title('Cell 4: adjust contractility amplitudes only')

def update(val):
    pos = scroll_slider.val
    yzoom = y_zoom.val
    xzoom = x_zoom.val
    ax.set_xlim(pos, pos+20*xzoom)
    fig.canvas.draw_idle
    ecc=np.median(peak_vals)+yzoom
    ax.set_ylim(-ecc, ecc)
    fig.canvas.draw_idle
    
# update function called using on_changed() function
scroll_slider.on_changed(update)
y_zoom.on_changed(update)
x_zoom.on_changed(update)

# Display the plot
plt.show()

def onclick(event):
    global peak_times,peak_vals,fig,peak_plot
    ix, iy, ib, ik, iax = event.xdata, event.ydata, event.button, event.key, event.inaxes
    if (ix!=None) & (iax==ax):
        if (event.button==1) & (ik==None) & (ix>0) & (ix>0) & (iy<np.max(peak_vals)*1.2): 
            peak_times,peak_vals,peak_plot=adjust_peak_amp(ix,iy,peak_times,peak_vals,peak_plot)
        elif (event.button==1)&(ik=='m'):
            peak_times,peak_vals,peak_plot=meap_dat_cell4(ix,iy,peak_times,peak_vals,peak_plot)
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        if not plt.fignum_exists(1):
            canvas.mpl_disconnect(cid)
            canvas.mpl_disconnect(cidk)
            
    return peak_times,peak_vals

def onarrow(e):
    global scroll_slider
    if e.key == "right":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]+.500)
    elif e.key == "left":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]-.500)
    else:
        return

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cidk = fig.canvas.mpl_connect('key_press_event', onarrow)

In [5]:
## CELL 5: OUTPUT DATA TO CSV ###
# This cell will model respiration and heart-rate out of the beatwise contractility estimates and output the following files:
# (1) Contractility estimates (XXX.csv)
# (2) Blood Pressure (XXX_bp.csv)
# (3) Respiration (XXX_resp.csv)
# (4) Event Codes (XXX_events.csv)

# These files can then be used for event-related analyses.

import pandas as pd
from sys_SCOT_Event_Related import regress_out,outsave_box
plt.close('all')

## output contractility
cont_dict['cont_peak_times']=peak_times
cont_dict['raw_contractility_peaks']=peak_vals
cont_dict=regress_out(cont_dict)

out_f = outsave_box(file_path[:-4]+'.csv')

out_dict={}
out_dict['time']=cont_dict['cont_peak_times']
out_dict['resid_contractility']=cont_dict['resid_contractility'] # renamed contractility >resid_contractility
out_dict['raw_contractility']=cont_dict['raw_contractility_peaks'] # added this output in case need to re-run pipeline later
pd.DataFrame(out_dict).to_csv(out_f)

# ouput event codes and corresponding timestamps
if 'events_s' in cont_dict: # dict will only contain "events_s" key if event channel present and selected in prepro
    out_f_event = out_f[:-4] + '_events.csv' #outsave_box(file_path[:-4]+'_events.csv')
    
    events = cont_dict['events_s']
    times = cont_dict['events_t']
    
    code_mat = []
    time_mat = []
    
    # cut out redundant events from oversampling
    for i in np.arange(np.shape(events)[0]): 
        if events[i] != 0 and events[i-1] != events[i]:       
            code_mat.append(int(events[i]))
            time_mat.append(round(times[i],3))
    
    # create df with event codes and timestamps and write to csv
    pd.DataFrame({'time':time_mat,'code':code_mat}).to_csv(out_f_event)
    
# output continuous blood pressure
if 'bp_s' in cont_dict: # if there's bp data present
    out_f_bp = out_f[:-4] + '_bp.csv'
    
    # grab bp times and values and downsample from 1000 Hz to 10 Hz
    #times=np.round(cont_dict['bp_t'][0:-1:100],1)
    #bp = np.round(cont_dict['bp_s'][0:-1:100],3)
    times=np.round(cont_dict['bp_t'][0:-1:10],2)
    bp = np.round(cont_dict['bp_s'][0:-1:10],3)

    pd.DataFrame({'time':times,'bp':bp}).to_csv(out_f_bp)

# output respiration
if 'raw_resp_s' in cont_dict: # if there's raw respiration data present
    out_f_resp = out_f[:-4] + '_resp.csv'
    
    # grab resp times and values and downsample from 1000 Hz to 10 Hz
    times=np.round(cont_dict['raw_resp_t'][0:-1:100],1)
    resp = np.round(cont_dict['raw_resp_s'][0:-1:100],3)
    #resp = cont_dict['raw_resp_s']

    pd.DataFrame({'time':times,'resp':resp}).to_csv(out_f_resp)